## Setup

In [ ]:
# GPU Configuration - Use P100 (GPU 1) for best performance
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # 0=GTX1080, 1=P100
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Verify GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPUs available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Standard imports
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.metrics import roc_auc_score

# Add functions to path
sys.path.insert(0, str(Path('.').resolve()))

# Import ensemble modules - use same optimization function as online path!
from functions.ensemble_initialization import create_data_splits
from functions.ensemble_stage2_training import optimize_and_update_config
from functions import ensemble_config

print("\nImports successful!")
print("Using SAME optimize_and_update_config() as online hill climbing")

### Install keras-tuner

In [ ]:
try:
    import keras_tuner
    print(f"keras-tuner {keras_tuner.__version__} already installed")
except ImportError:
    print("Installing keras-tuner...")
    !pip install keras-tuner
    import keras_tuner
    print(f"keras-tuner {keras_tuner.__version__} installed")

### Configuration

In [ ]:
# Configuration
RANDOM_STATE = 315
LABEL = 'diagnosed_diabetes'

# Hyperparameter search settings (GPU-optimized)
MAX_TRIALS = 30  # ~15-20 min on P100
EXECUTIONS_PER_TRIAL = 3  # Statistical confidence

# Database path for loading ensemble state
DB_PATH = '../data/ensemble_training.db'

print(f"Configuration:")
print(f"  Max trials: {MAX_TRIALS}")
print(f"  Executions per trial: {EXECUTIONS_PER_TRIAL}")
print(f"  Database: {DB_PATH}")
print(f"  Estimated time: ~15-30 minutes on P100 GPU")

## Data Preparation

In [ ]:
# Load training data
train_df_path = 'https://gperdrizet.github.io/FSA_devops/assets/data/unit3/diabetes_prediction_train.csv'
train_df = pd.read_csv(train_df_path)
train_df.drop_duplicates(inplace=True)

print(f'Training data shape: {train_df.shape}')
print(f'Class distribution:')
print(train_df[LABEL].value_counts(normalize=True))

In [ ]:
# Create 60/20/20 split (same as hill climbing)
X_train_pool, X_val_s1, X_val_s2, y_train_pool, y_val_s1, y_val_s2 = create_data_splits(
    train_df, LABEL, RANDOM_STATE
)

print(f"\nData splits (same as hill climbing):")
print(f"  X_train_pool: {X_train_pool.shape} (60%)")
print(f"  X_val_s1: {X_val_s1.shape} (20%)")
print(f"  X_val_s2: {X_val_s2.shape} (20%)")

## Load Stage 1 Models from Database

Query the database to find the latest complete batch (10 models) and load those models.

In [ ]:
import joblib
import sqlite3
from glob import glob

# Query database to find latest batch of 10 models
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Get the latest ensemble_id that has at least 10 models
query = """
SELECT ensemble_id, MAX(num_models) as max_models, MAX(iteration_num) as last_iter
FROM ensemble_log
WHERE num_models >= 10
GROUP BY ensemble_id
ORDER BY last_iter DESC
LIMIT 1
"""

result = cursor.execute(query).fetchone()

if result is None:
    print("No batch with 10+ models found in database.")
    print("Checking current progress...")
    
    # Check current state
    query2 = """
    SELECT ensemble_id, num_models, iteration_num
    FROM ensemble_log
    ORDER BY iteration_num DESC
    LIMIT 1
    """
    current = cursor.execute(query2).fetchone()
    
    if current:
        print(f"  Latest state: {current[1]} models at iteration {current[2]}")
        print(f"  Ensemble ID: {current[0]}")
    
    conn.close()
    raise ValueError("Need at least 10 models for batch 1 optimization. Continue hill climbing first.")

ensemble_id, num_models, last_iter = result
conn.close()

print(f"Found complete batch:")
print(f"  Ensemble ID: {ensemble_id}")
print(f"  Total models: {num_models}")
print(f"  Last iteration: {last_iter}\n")

# Extract run directory from ensemble_id (format: run_YYYYMMDD_HHMMSS_batch_X)
run_dir = '_'.join(ensemble_id.split('_')[:3])  # e.g., run_20251208_045148
models_path = f"../models/{run_dir}/ensemble_stage1_models"

print(f"Loading models from: {models_path}\n")

# Load first 10 models (founder + models 1-9)
model_files = [f"{models_path}/founder_model.joblib"]
for i in range(1, 10):
    model_files.append(f"{models_path}/model_{i}.joblib")

stage1_models = []
for model_file in model_files:
    if not Path(model_file).exists():
        print(f"Warning: {model_file} not found, skipping...")
        continue
    
    model_name = Path(model_file).stem
    model = joblib.load(model_file)
    
    # Evaluate on validation set
    pred = model.predict_proba(X_val_s1)[:, 1]
    auc = roc_auc_score(y_val_s1, pred)
    
    stage1_models.append(model)
    print(f"  {model_name}: AUC = {auc:.6f}")

print(f"\n{len(stage1_models)} Stage 1 models loaded for batch 1!")

## Run Hyperparameter Optimization

**Uses SAME `optimize_and_update_config()` function as online hill climbing!**

This ensures:
- Identical search space
- Same data preparation (conservative 95/5 split)
- Same architecture builders (pyramid/funnel)
- Results directly applicable to production

In [ ]:
print("=" * 80)
print("RUNNING STAGE 2 DNN HYPERPARAMETER OPTIMIZATION")
print("Using SAME function as online hill climbing: optimize_and_update_config()")
print("=" * 80)

# Run optimization - this will update ensemble_config.STAGE2_DNN_CONFIG in-memory
optimized_config = optimize_and_update_config(
    ensemble_models=stage1_models,
    X_val_s1=X_val_s1,
    y_val_s1=y_val_s1,
    X_val_s2=X_val_s2,
    y_val_s2=y_val_s2,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL
)

print("\n" + "=" * 80)
print("OPTIMIZATION COMPLETE")
print("=" * 80)

## Results

In [ ]:
# Display optimized configuration
import json

print("\nOptimized Stage 2 DNN Configuration:")
print("=" * 80)
print(json.dumps(optimized_config, indent=2))
print("=" * 80)

In [ ]:
# Extract key hyperparameters
layers = optimized_config['architecture']['hidden_layers']
lr = optimized_config['training']['learning_rate']

print("\nKey Hyperparameters:")
print("-" * 40)
print(f"Number of layers: {len(layers)}")
print(f"Layer configuration:")
for i, layer in enumerate(layers, 1):
    print(f"  Layer {i}: {layer['units']} units, dropout={layer['dropout']:.3f}")
print(f"Learning rate: {lr:.6f}")
print("-" * 40)

## Copy-Paste Configuration

Use this configuration in `ensemble_config.py` to apply optimized hyperparameters.

In [ ]:
print("\n" + "=" * 80)
print("COPY-PASTE CONFIGURATION FOR ensemble_config.py")
print("=" * 80)
print()
print("# Stage 2 DNN Configuration (optimized via Keras Tuner)")
print(f"# Optimized on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"# Models used: {len(stage1_models)} (from {LATEST_RUN})")
print(f"# Trials: {MAX_TRIALS} × {EXECUTIONS_PER_TRIAL} executions")
print()
print("STAGE2_DNN_CONFIG = " + json.dumps(optimized_config, indent=4))
print()
print("=" * 80)

## Summary

✅ **Test notebook complete!**

### What we did:

1. ✅ Loaded training data (60/20/20 split)
2. ✅ Loaded actual Stage 1 models from batch 1
3. ✅ Ran **`optimize_and_update_config()`** (same as online hill climbing)
4. ✅ Displayed optimized hyperparameters
5. ✅ Generated copy-paste config for `ensemble_config.py`

### Key points:

- **Shared code path**: Uses identical optimization function as online training
- **Real data**: Uses actual Stage 1 models from hill climbing run
- **GPU-optimized**: ~15-30 min on P100 GPU
- **Focused search**: pyramid/funnel architectures, 2-3 layers, 64-256 units
- **Conservative split**: 95% training (266k), 5% validation (14k)

### Next steps:

1. Copy the `STAGE2_DNN_CONFIG` above into `ensemble_config.py`
2. Run full hill climbing with GPU enabled
3. Architecture will be automatically optimized at batches 10, 20, 30+
4. Compare performance against baseline